# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution

## Answer 1

In [3]:
def sum_generator(limit):
    n = 1
    sm = []
    while n <= limit:
        sm.append(n ** 0.5)
        yield sum(sm)
        n += 1

# Example usage:
limit = 5
generator = sum_generator(limit)

for sum_value in generator:
    print(sum_value)

1.0
2.414213562373095
4.146264369941973
6.146264369941973
8.382332347441762


## Answer 2

In [4]:
# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


## Answer *3*

In [5]:
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

In [6]:
import duckdb
import dlt

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='data')
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

### setup

### load first time

In [7]:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset data
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708044348.7173808 is LOADED and contains no failed jobs


In [8]:
conn.sql('SELECT * FROM people')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708044348.7173808 │ LLhi7D7E7LZDMg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708044348.7173808 │ W7bKrlEwoH1gdg │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1708044348.7173808 │ Cl9k94kHb+HQlQ │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1708044348.7173808 │ VrEfqtd1eNTyTQ │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1708044348.7173808 │ 3VvXp+wFmLT/9A │ NULL       │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┴────────────┘

In [9]:
conn.sql('SELECT SUM(age) FROM people')

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

### add and update

In [10]:
info = generators_pipeline.run(people_2,
										table_name="people",
										write_disposition="append",
                    )
print(info)
conn.sql('SELECT * FROM people')

Pipeline dlt_colab_kernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset data
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708044349.5633864 is LOADED and contains no failed jobs


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708044348.7173808 │ LLhi7D7E7LZDMg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708044348.7173808 │ W7bKrlEwoH1gdg │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1708044348.7173808 │ Cl9k94kHb+HQlQ │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1708044348.7173808 │ VrEfqtd1eNTyTQ │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1708044348.7173808 │ 3VvXp+wFmLT/9A │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1708044349.5633864 │ YPZt5YFbrdOK+A │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1708044349.5633864 │ 1VPlvfCD5ZpbEg │ Job_4      │
│     5 │ 

In [11]:
conn.sql('SELECT SUM(age) FROM people')

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

## Answer 4

In [12]:
import duckdb
import dlt

generators_pipeline_merge = dlt.pipeline(destination='duckdb', dataset_name='data_merge')
conn = duckdb.connect(f"{generators_pipeline_merge.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline_merge.dataset_name}'")

### setup

### load first time

In [14]:
# the outcome metadata is returned by the load and we can inspect it by printing it.
info = generators_pipeline_merge.run(people_1(),
										table_name="people",
										write_disposition="replace")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset data_merge
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708044362.4206784 is LOADED and contains no failed jobs


In [15]:
conn.sql('SELECT * FROM people')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708044362.4206784 │ zypbBUP+3rOh9A │
│     2 │ Person_2 │    27 │ City_A  │ 1708044362.4206784 │ FTn2GutzMJc+MQ │
│     3 │ Person_3 │    28 │ City_A  │ 1708044362.4206784 │ I9pQtUyI4N3hFA │
│     4 │ Person_4 │    29 │ City_A  │ 1708044362.4206784 │ yyX5AySWW83eKw │
│     5 │ Person_5 │    30 │ City_A  │ 1708044362.4206784 │ XqlVrfKGo4hFew │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┘

In [16]:
conn.sql('SELECT SUM(age) FROM people')

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

### add and update

In [17]:
info = generators_pipeline_merge.run(people_2,
										table_name="people",
										write_disposition="merge",
                    primary_key="id"
                    )
print(info)
conn.sql('SELECT * FROM people')

Pipeline dlt_colab_kernel_launcher load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset data_merge
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708044401.162202 is LOADED and contains no failed jobs


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708044362.4206784 │ zypbBUP+3rOh9A │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708044362.4206784 │ FTn2GutzMJc+MQ │ NULL       │
│     5 │ Person_5 │    35 │ City_B  │ 1708044401.162202  │ eObA+MV6YcACQw │ Job_5      │
│     7 │ Person_7 │    37 │ City_B  │ 1708044401.162202  │ QdvhF2Q3q38Mlg │ Job_7      │
│     4 │ Person_4 │    34 │ City_B  │ 1708044401.162202  │ bZRTUx1/aLu+Kg │ Job_4      │
│     8 │ Person_8 │    38 │ City_B  │ 1708044401.162202  │ glzpTXB/ISA47Q │ Job_8      │
│     3 │ Person_3 │    33 │ City_B  │ 1708044401.162202  │ AeckgGoNKg1NgA │ Job_3      │
│     6 │ 

In [18]:
conn.sql('SELECT SUM(age) FROM people')

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘